# guidelines

TODO : import whenever needed, not centralized

states https://pytorch.org/tutorials/beginner/saving_loading_models.html

# Introduction 

## Aim

## Data

In [ ]:
# download

# Import (Remove section later on)

In [ ]:
from adversary import attack, protect
from net import Net
import numpy as np
from torch.optim import Optimizer
import torch
from training import training, testing, accuracy, tune_optimizer
from minibatch import MiniBatchOptimizer
from adam import AdamOptimizer
import matplotlib.pyplot as plt
from data_utils import get_mnist, build_data_loaders
import json
from pathlib import Path
import random
import pandas as pd

## Setup

In [ ]:
# flags
# use cuda = True
# tune_hyperparam = True

In [ ]:
# device = 

In [ ]:
use_cuda = True
device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')
print("Device chosen is {}".format(device))
train_dataset, test_dataset = get_mnist(normalize=True)
epsilons = np.arange(0, 0.5, 0.05)
criterion = torch.nn.CrossEntropyLoss()
epochs = 10
batch_size = 16

# Model

In [ ]:
# import the model

# net = fhreifg
# initial_state = net.get_state()

# Hyperparameter tuning

In [ ]:
# parameters that are in common batch size, epochs

In [ ]:
hyperparameter_tune = False

## Adam

In [ ]:
net_tune = Net().to(device)
adam_tune = AdamOptimizer(net_tune.parameters()) 
fp = './res/adam_tuning.json'


In [ ]:
if not hyperparameter_tune:
    results = []
else:
    results = tune_optimizer(
    net_tune,
    train_dataset.data,
    train_dataset.targets,
    criterion,
    accuracy,
    device,
    AdamOptimizer,
    epochs=10,
    search_grid={
        'lr': np.linspace(0.00001, 0.01, 5),
        'beta1':  np.linspace(0.1, 0.9, 5),
        'beta2': np.linspace(0.5, 0.999, 5),
        'weight_decay': np.linspace(0.0001, 0.1, 4),
        'epsilon': np.linspace(1e-10, 1e-8, 3),
    }
    batch_size=16
)

In [ ]:
if Path(fp).exists():
    with open(fp, 'r') as f:
        old_results = json.load(f)

    results = old_results + results

with open(fp, 'w') as f:
    json.dump(results, f, indent=2)

# Select Best Hyperparamters
with open(fp, 'r') as f:
        old_results = json.load(f)

In [ ]:
def find_best_params_adam(adam_tuning):
    best_params = dict()
    best_params['loss_test'] = float('inf')
    best_params['acc_test'] = -float('inf')

        
    #IF STD, divide metrics by std
    #Wait for the new tuning
    for item in adam_tuning:
        if best_params['acc_test'] < item['metric_test'] or best_params['loss_test'] > item['loss_test']:
            best_params['loss_train'] = item['loss_train']
            best_params['acc_train'] = item['metric_train']
            best_params['acc_test'] = item['metric_test']
            best_params['loss_test'] = item['loss_test']
            best_params['lr'] = item['lr']
            best_params['beta1'] = item['beta1']
            best_params['beta2'] = item['beta2']
            best_params['weight_decay'] = item['weight_decay']
            best_params['epsilon'] = item['epsilon']
            
    return best_params
best_adam_params = find_best_params_adam(adam_tuning)
print_adam_stats(best_adam_params)

## Nesterov

## Minibatch

In [ ]:
net_tune = Net().to(device)
mini_opt_tune = MiniBatchOptimizer(net_tune.parameters()) # Just using defaults
dec_lr_set =  [0]*1 + [1]*1
random.shuffle(dec_lr_set)
fp = './res/mini_tuning.json'
if not hyperparamter_tune:
    results = []

In [ ]:
if hyperparamter_tune:
    results = tune_optimizer(
    net_tune,
    train_dataset.data,
    train_dataset.targets,
    criterion,
    accuracy,
    device,
    MiniBatchOptimizer,
    epochs=10,
    search_grid={
        'lr': np.linspace(0.00001, 0.01, 5),
        'decreasing_lr': dec_lr_set,
    }, 
    batch_size=16
)

In [ ]:
if Path(fp).exists():
    with open(fp, 'r') as f:
        old_results = json.load(f)

    results = old_results + results

with open(fp, 'w') as f:
    json.dump(results, f, indent=2)

# Select Best Hyperparamters
with open(fp, 'r') as f:
        old_results = json.load(f)

In [ ]:
df_analysis = pd.DataFrame(results)
best_acc = 0.0
for index, row in df_analysis.iterrows():    
        trial_acc = row["metric_test"]
        if trial_acc > best_acc:
            best_acc = trial_acc
            learning_rate = round(row["lr"], 6)
            decreasing_lr = row["decreasing_lr"]

print("Best Accuracy was {}% with Learning Rate {} and Decreasing LR: {}".format(100*best_acc, learning_rate, decreasing_lr))


In [ ]:
# lst_optimizer = {'name': function_optimizer}

## Comparison

# Attack on naive model



## Train naive models

### Minibatch (for now, loop later)

In [ ]:
net_naive = Net().to(device)
train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)

In [ ]:
mini_opt_naive = MiniBatchOptimizer(net_naive.parameters(), lr=learning_rate, decreasing_lr=decreasing_lr)
loss_train, acc_train = training(net_naive, train_loader, mini_opt_naive, criterion, accuracy, epochs=epochs, device=device)
loss_test, acc_test = testing(net_naive, test_loader, criterion, accuracy, device=device)

## Adam

In [ ]:
net_naive_adam = Net().to(device)
train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)

In [ ]:
adam_opt_naive = AdamOptimizer(net_naive.parameters(), lr=best_adam_params['lr'], beta1=best_adam_params['beta1'],beta2=best_adam_params['beta2'],weight_decay=best_adam_params['weight_decay'],epsilon=best_adam_params['epsilon'])
loss_train, acc_train = training(net_naive, train_loader, adam_opt_naive, criterion, accuracy, epochs=epochs, device=device)
loss_test, acc_test = testing(net_naive, test_loader, criterion, accuracy, device=device)

## Attack naive models

In [ ]:
# use the lst_optimizer
# Only one optimizer used in this part?

### Minibatch (for now, loop later)

In [ ]:
accuracy_naive= []
losses_naive= []

for eps in epsilons:
    loss_attack, acc_attack  = attack(net_naive, criterion, test_loader, epsilon=eps, device=device)
    accuracy_naive.append(acc_attack)
    losses_naive.append(loss_attack)

### Adam

In [ ]:
accuracy_naive_adam= []
losses_naive_adam= []

for eps in epsilons:
    loss_attack, acc_attack  = attack(net_naive_adam, criterion, test_loader, epsilon=eps, device=device)
    accuracy_naive_adam.append(acc_attack)
    losses_naive_adam.append(loss_attack)

## Comparison

# Attack on robust model

## Train robust models

### Minibatch (for now, loop later)

In [ ]:
robust_net = Net().to(device)
protect_epochs = epochs
protect_lr = learning_rate
protect_bz = batch_size
protect_dec_lr = decreasing_lr
prot_train_loader, prot_test_loader = build_data_loaders(train_dataset, test_dataset, protect_bz)
mini_opt_proc = MiniBatchOptimizer(robust_net.parameters(), lr=protect_lr, decreasing_lr=protect_dec_lr)

In [ ]:
robust_net = protect(robust_net, mini_opt_proc, criterion, prot_train_loader, prot_test_loader, device=device, epochs=protect_epochs)

## Adam

In [ ]:
robust_net = Net().to(device)
protect_epochs = epochs
protect_lr = learning_rate
protect_bz = batch_size

prot_train_loader, prot_test_loader = build_data_loaders(train_dataset, test_dataset, protect_bz)
adam_opt_proc = AdamOptimizer(net_naive.parameters(), lr=best_adam_params['lr'], beta1=best_adam_params['beta1'],beta2=best_adam_params['beta2'],weight_decay=best_adam_params['weight_decay'],epsilon=best_adam_params['epsilon'])

In [ ]:
robust_net_adam = protect(robust_net, adam_opt_proc, criterion, prot_train_loader, prot_test_loader, device=device, epochs=protect_epochs)

## Attack robust models

### Minibatch (for now, loop later)

In [ ]:
accuracy_robust = []
losses_robust = []
# This should be the first term test_loader is used
for eps in epsilons:
    loss_attack, acc_attack = attack(robust_net, criterion, prot_test_loader, eps, device=device)
    accuracy_robust.append(acc_attack)
    losses_robust.append(loss_attack)

### Adam

In [ ]:
accuracy_robust_adam = []
losses_robust_adam = []
# This should be the first term test_loader is used
for eps in epsilons:
    loss_attack, acc_attack = attack(robust_net_adam, criterion, prot_test_loader, eps, device=device)
    accuracy_robust_adam.append(acc_attack)
    losses_robust_adam.append(loss_attack)

## Comparison

# Comparative analysis

### Minibatch (for now)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracy_naive, "*-", c='blue', label='Naive Model')
plt.plot(epsilons, accuracy_robust, "*-", c='orange', label='Robust Model')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, 0.5, step=0.05))

plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.legend();

Lots of plots

* diff naive vs robust (algo as hue)